In [ ]:
!pip install opencv-python

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import os
import pathlib
import random as rn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import argparse


In [ ]:
from PIL import Image

# Open the PPM image file
image = Image.open('00002.ppm')

image

In [ ]:
def equalize(img):
    #print(type(img), img.shape)
    img1=np.uint8(cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX))
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img = clahe.apply(img1)
    img = cv2.equalizeHist(img)
    img = tf.expand_dims(img, axis=-1)
    img = np.array(img)
    img = img.astype(np.float64)
    #print(type(img), img.shape)
    return img

In [ ]:
data_dir = 'C:/Users/senti/Desktop/GTSDB'
train_path = 'C:/Users/senti/Desktop/GTSDB/Train'
test_path = 'C:/Users/senti/Desktop/GTSDB/Test/'
height = 32
width = 32

In [ ]:
batch_size = 150
seed = 42

In [ ]:
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.transforms.functional import equalize

# Image transformations for train and test datasets
transform = transforms.Compose([
    transforms.Grayscale(),  # Convert to grayscale
    transforms.Resize((height, width)),  # Resize images
    transforms.Lambda(lambda img: equalize(img)),  # Equalize images
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.5], std=[0.5]),  # Rescale to range [-1, 1]
])

# Load the dataset
dataset = datasets.ImageFolder(train_path, transform=transform)

# Split into training and validation sets (80/20 split)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


In [ ]:
# For train dataset
train_labels = [label for _, label in train_dataset]

# For validation dataset
val_labels = [label for _, label in val_dataset]

In [ ]:
list_set = set(train_labels)
unique_list = (list(list_set))

print(unique_list)
unique_list.append(43)

In [ ]:
fig,ax=plt.subplots(3,4)
fig.set_size_inches(16,12)
img,y = train_dataset.next()
for i in range(3):
    for j in range (4):
        l=rn.randint(0,batch_size-1)
        label = classes[int(list(train_dataset.class_indices.keys())[np.argmax(y[l])])]
        ax[i,j].imshow(img[l])
        ax[i,j].set_title(label)

plt.tight_layout()

In [ ]:
import argparse
import json
import tensorboard
import tensorboardX
import os
import argparse
import json
import os
import numpy as np
import torch
import torch.nn as nn
import nni
from nni.nas.nn.pytorch import ModelSpace, LayerChoice, MutableConv2d, MutableBatchNorm2d, MutableReLU
from pytorch_lightning import Trainer
from nni.nas.evaluator.pytorch import Lightning, ClassificationModule, Trainer
from nni.nas.experiment import NasExperiment
from nni.nas.space import model_context
from nni.nas.hub.pytorch import DARTS
from nni.nas.strategy import DARTS as DartsStrategy
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms
from torchvision.datasets import CIFAR10
from nni.nas.experiment import NasExperiment
from nni.nas.evaluator import FunctionalEvaluator
from nni.nas.evaluator import FunctionalEvaluator
import nni.nas.strategy as strategy
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
#from ops import AvgPool,DilConv,SepConv
import genotypes
from pytorch_lightning.callbacks import ModelCheckpoint
torch.set_float32_matmul_precision('medium')
from tqdm import tqdm
from nni.nas.nn.pytorch import LayerChoice, ModelSpace,ValueChoice
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from pytorch_lightning import LightningModule, Trainer
from torchvision import datasets, transforms
from nni.nas.evaluator.pytorch import Classification

In [ ]:
import torch
import torch.nn.functional as F

class DropPath(nn.Module):
    def __init__(self, drop_prob=0.0):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        if self.drop_prob == 0. or not self.training:
            return x
        keep_prob = 1 - self.drop_prob
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # broadcast across channels
        random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
        random_tensor.floor_()  # binarize
        return x.div(keep_prob) * random_tensor  # scale output to maintain expectations

    def set_drop_prob(self, drop_prob):
        self.drop_prob = drop_prob

In [ ]:
class SepConv(nn.Module):
    def __init__(self, C_in, C_out, kernel_size, stride, padding, affine=True):
        super(SepConv, self).__init__()
        self.op = nn.Sequential(
            nn.Conv2d(C_in, C_in, kernel_size=kernel_size, stride=stride, padding=padding, groups=C_in),  
            nn.Conv2d(C_in, C_in, kernel_size=1, padding=0),  
            nn.Conv2d(C_in, C_in, kernel_size=kernel_size, stride=stride, padding=padding, groups=C_in), 
            nn.Conv2d(C_in, C_out, kernel_size=1, padding=0),  
            nn.BatchNorm2d(C_out, affine=affine),
            nn.ReLU(inplace=True)
        )

In [ ]:

class CustomDARTSSpaceK3(ModelSpace):
    def __init__(self, input_channels, channels, num_classes, layers, verbose):
        super(CustomDARTSSpaceK3, self).__init__()
        #self.first_iter = True
        self.layers = nn.ModuleList()
        self.drop_path_prob = 0.0
        self.drop_path = DropPath(self.drop_path_prob)
        self.preliminary_layer = nn.Conv2d(3, 8, kernel_size=3, padding=0, bias=False)
        self.verbose = verbose

        self.layer1_conv1=nn.Conv2d(8, 8, kernel_size=3, stride=1, padding=1, groups=8)
        self.layer1_conv2=nn.Conv2d(8, 8, kernel_size=1, padding=0)
        self.layer1_conv3=nn.Conv2d(8, 8, kernel_size=3, stride=1, padding=1, groups=8)
        self.layer1_conv4=nn.Conv2d(8, 64, kernel_size=1, padding=0)
        self.layer1_bn=nn.BatchNorm2d(64, affine=True)
        self.layer1_relu=nn.ReLU(inplace=True)

        
        self.layer2_conv1=nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, groups=64)
        self.layer2_conv2=nn.Conv2d(64, 64, kernel_size=1, padding=0)
        self.layer2_conv3=nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, groups=64)
        self.layer2_conv4=nn.Conv2d(64, 2, kernel_size=1, padding=0)
        self.layer2_bn=nn.BatchNorm2d(2, affine=True)
        self.layer2_relu=nn.ReLU(inplace=True)  

        
        self.layer3_conv1=nn.Conv2d(2, 2, kernel_size=3, stride=1, padding=1, groups=2)
        self.layer3_conv2=nn.Conv2d(2, 2, kernel_size=1, padding=0)
        self.layer3_conv3=nn.Conv2d(2, 2, kernel_size=3, stride=1, padding=1, groups=2)
        self.layer3_conv4=nn.Conv2d(2, 2, kernel_size=1, padding=0)
        self.layer3_bn=nn.BatchNorm2d(2, affine=True)
        self.layer3_relu=nn.ReLU(inplace=True)  

        
        self.layer4_conv1=nn.Conv2d(2, 2, kernel_size=3, stride=1, padding=1, groups=2)
        self.layer4_conv2=nn.Conv2d(2, 2, kernel_size=1, padding=0)
        self.layer4_conv3=nn.Conv2d(2, 2, kernel_size=3, stride=1, padding=1, groups=2)
        self.layer4_conv4=nn.Conv2d(2, 2, kernel_size=1, padding=0)
        self.layer4_bn=nn.BatchNorm2d(2, affine=True)
        self.layer4_relu=nn.ReLU(inplace=True)  

        
        self.layer5_conv1=nn.Conv2d(2, 2, kernel_size=3, stride=1, padding=1, groups=2)
        self.layer5_conv2=nn.Conv2d(2, 2, kernel_size=1, padding=0)
        self.layer5_conv3=nn.Conv2d(2, 2, kernel_size=3, stride=1, padding=1, groups=2)
        self.layer5_conv4=nn.Conv2d(2, 22, kernel_size=1, padding=0)
        self.layer5_bn=nn.BatchNorm2d(22, affine=True)
        self.layer5_relu=nn.ReLU(inplace=True)  

        
        self.pool = nn.AdaptiveAvgPool2d((2, 2))
        self.fc1 = nn.Linear(198, 128) 
        self.fc2 = nn.Linear(128, 64) 
        self.fc3 = nn.Linear(64, 32)  
        self.relu = nn.ReLU()
        self.classifier = nn.Linear(32, num_classes)
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, 0, 0.01)
                init.constant_(m.bias, 0)
    def forward(self, x):

        x = self.preliminary_layer(x)
        # Unroll layer1
        x = self.layer1_conv1(x)
        x = self.layer1_conv2(x)
        x = self.layer1_conv3(x)
        x = self.layer1_conv4(x)
        x = self.layer1_bn(x)
        x = self.layer1_relu(x)
        x = self.drop_path(x)  
        
        # Unroll layer2
        x = self.layer2_conv1(x)
        x = self.layer2_conv2(x)
        x = self.layer2_conv3(x)
        x = self.layer2_conv4(x)
        x = self.layer2_bn(x)
        x = self.layer2_relu(x)
        x = self.drop_path(x) 
        
        # First AvgPool after layer2
        x = nn.AvgPool2d(kernel_size=2, stride=2)(x)
        
        # Unroll layer3
        x = self.layer3_conv1(x)
        x = self.layer3_conv2(x)
        x = self.layer3_conv3(x)
        x = self.layer3_conv4(x)
        x = self.layer3_bn(x)
        x = self.layer3_relu(x)
        x = self.drop_path(x)  
        
        # Unroll layer4
        x = self.layer4_conv1(x)
        x = self.layer4_conv2(x)
        x = self.layer4_conv3(x)
        x = self.layer4_conv4(x)
        x = self.layer4_bn(x)
        x = self.layer4_relu(x)
        x = self.drop_path(x)  
        
        # Second AvgPool after layer4
        x = nn.AvgPool2d(kernel_size=2, stride=2)(x)
        
        # Unroll layer5
        x = self.layer5_conv1(x)
        x = self.layer5_conv2(x)
        x = self.layer5_conv3(x)
        x = self.layer5_conv4(x)
        x = self.layer5_bn(x)
        x = self.layer5_relu(x)
        x = self.drop_path(x)  
        
        x =  nn.AvgPool2d(kernel_size=2, stride =2)(x)
        if self.verbose == 1 :
            print(f'After adaptive pooling: {x.shape}')
        
        x = torch.flatten(x, 1)
        if self.verbose == 1 :
            print(f'After flattening: {x.shape}')
        
        x = self.fc1(x)
        x= self.relu(x)
        if self.verbose == 1 :
            print(f'After fc1: {x.shape}')
        x = self.fc2(x)
        x= self.relu(x)
        if self.verbose == 1 :
            print(f'After fc2: {x.shape}')
        x = self.fc3(x)
        x= self.relu(x)
        if self.verbose == 1 :
            print(f'After fc3: {x.shape}')
        
        x = self.classifier(x)
        if self.verbose == 1 :
            print(f'After classifier: {x.shape}')
        #self.first_iter = False
        return x


    def set_drop_path_prob(self, drop_path_prob):
        self.drop_path_prob = drop_path_prob
        self.drop_path.set_drop_prob(drop_path_prob)


In [ ]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset, random_split
import torch

def get_cifar10_dataset(train: bool = True, cutout: bool = False):
    CIFAR_MEAN = [0.49139968, 0.48215827, 0.44653124]
    CIFAR_STD = [0.24703233, 0.24348505, 0.26158768]

    if train:
        transform_list = [
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ToTensor(),
            transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
        ]
        # Define or remove cutout_transform if not needed
        if cutout:
            transform_list.append(cutout_transform)  
        transform = transforms.Compose(transform_list)
    else:
        # No data augmentation for validation/testing
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
        ])

    dataset = datasets.CIFAR10(root='./data', train=train, download=True, transform=transform)
    return dataset

# Hyperparameters
batch_size = 64
validation_split = 0.5

# Get the full training dataset
train_data = get_cifar10_dataset(train=True)

# Split the dataset into training and validation sets
train_size = int(len(train_data) * (1 - validation_split))
valid_size = len(train_data) - train_size

train_dataset, valid_dataset = random_split(train_data, [train_size, valid_size])

# DataLoader for training set
train_loader = DataLoader(
    train_dataset, batch_size=batch_size,
    shuffle=True, pin_memory=True, num_workers=6
)

# DataLoader for validation set
valid_loader = DataLoader(
    valid_dataset, batch_size=batch_size,
    shuffle=False, pin_memory=True, num_workers=18
)

# If needed, load the test set
test_data = get_cifar10_dataset(train=False)
test_loader = DataLoader(
    test_data, batch_size=batch_size,
    shuffle=False, pin_memory=True, num_workers=18
)


In [ ]:
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn

# Initialize optimizer, criterion, and scheduler (optional)
optimizer = optim.Adam(
    model.parameters(),
    lr=1e-3,  # Reduced learning rate for more stable training
    betas=(0.9, 0.999),
    eps=1e-07,
    weight_decay=1e-4
)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)  # Optional scheduler
criterion = nn.CrossEntropyLoss()

def train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Move the model to the device (GPU if available)
    model.train()  # Set the model to training mode
    
    for epoch in range(num_epochs):
        # Dynamically set DropPath probability (if your model supports it)
        drop_prob = 0.2 * (epoch / num_epochs)  # Adjust the scale as needed
        model.set_drop_path_prob(drop_prob)
        
        running_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        optimizer.zero_grad()  # Zero the gradients

        for i, (inputs, labels) in enumerate(progress_bar):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backward pass
            
            # Gradient clipping to prevent instability
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            # Gradient accumulation (if using)
            if (i + 1) % 4 == 0:
                optimizer.step()
                optimizer.zero_grad()

            running_loss += loss.item()

            # Update tqdm description with current loss
            progress_bar.set_postfix(loss=running_loss / (i+1))

        # Step scheduler (optional)
        scheduler.step()

        if epoch%9 == 0:
            val_accuracy = evaluate_model(model, valid_loader, device)
            print(f"Validation accuracy after epoch {epoch+1}: {val_accuracy * 100:.2f}%")

    print('Finished Training')

def evaluate_model(model, valid_loader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            correct += preds.eq(labels).sum().item()
    
    return correct / len(valid_loader.dataset)

# Example of running the training
train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=100)


In [ ]:
def validate_model(model, valid_loader, criterion):
    model.eval()
    total = 0
    correct = 0
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.cuda(), labels.cuda()  # Move data to GPU if available

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Loss: {running_loss / len(valid_loader):.4f}')
    print(f'Validation Accuracy: {100 * correct / total:.2f}%')

validate_model(model, valid_loader, criterion)